In [317]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
import pandas as pd
import numpy as np

In [318]:
posts = pd.read_csv('shuffled_train.csv')


In [4]:
from sklearn.naive_bayes import GaussianNB

In [319]:
import string
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.cross_validation import KFold
from nltk.tokenize import sent_tokenize

from __future__ import division

import numpy as np

from scipy import sparse

from sklearn.svm import SVC
from sklearn.utils import check_random_state, safe_indexing



In [320]:
lemmatizer = WordNetLemmatizer()

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

## Removing HTML tags
def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

## Removing URL
def strip_URL(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)

## Removing Hashtags and @tags 
def strip_all_tags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

## Removing special characters 
def strip_special_chars(text):
    return re.sub('\W+',' ', text)

## Removing extra white spaces
def strip_extra_white_spaces(text):
    return re.sub(' +',' ',text).strip()

## Removing extra new lines
def strip_extra_new_lines(text):
    return re.sub("\n+","\n", text)

def inverse_transform(l):
        sent = ''
        for word in l:
            sent += word+" "
        return [sent]


def tokenize(document):
    document = cleanhtml(document)
    document = strip_URL(document)
    document = strip_all_tags(document)
    document = strip_special_chars(document)
    document = strip_extra_white_spaces(document)
    document = strip_extra_new_lines(document)
    stopwords_custom = pd.read_csv("stopwords1.csv")
    stopwords_custom = stopwords_custom.ix[:,0].map(lambda x:x.lower())
        
    # Break the document into sentences
    for sent in sent_tokenize(document):
        # Break the sentence into part of speech tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Apply preprocessing to the token
            token = token.lower() 

            # If stopword, ignore token and continue
            if token.lower in stopwords_custom.tolist():
                continue

            # If punctuation, ignore token and continue
            if all(char in string.punctuation for char in token):
                  continue
            
            yield token

            # Lemmatize the token and yield
#             lemma = lemmatize(token, tag)
#             yield lemma

def lemmatize(token, tag):
    tag = {
       'N': wn.NOUN,
       'V': wn.VERB,
       'R': wn.ADV,
       'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    #return lemmatizer.lemmatize(token, tag)
    
def preprocessing(X):
    #tokens_dict = {}
    tokens_list = []
    count = 0
    for doc in X:
        tokens = list(tokenize(doc))
        # tokens_list
        tokens_list.append(tokens)
# tokens_dict[count] = inverse_transform(tokens) count += 1
    return tokens_list


In [321]:
words= preprocessing(posts['Tokenized.Sentences'])
words

[['i', 'will', 'try', 'and', 'stay', 'in', 'touch'],
 ['wishing', 'you', 'both', 'well', 'deb', 'w', 'nct02739386'],
 ['any',
  'suggestions',
  'on',
  'how',
  'to',
  'get',
  'your',
  'liver',
  'to',
  'cooperate',
  'with',
  'the',
  'meds'],
 ['the',
  'next',
  'day',
  'we',
  'saw',
  'him',
  'due',
  'to',
  'the',
  'colitis',
  'and',
  'had',
  'an',
  'emergency',
  'consult',
  'with',
  'dr'],
 ['the',
  'side',
  'effects',
  'were',
  'not',
  'as',
  'bad',
  'for',
  'him',
  'as',
  'they',
  'were',
  'on',
  'the',
  'ipilimumab',
  'initially',
  'there',
  'was',
  'a',
  'great',
  'amount',
  'of',
  'shrinkage',
  'from',
  'the',
  'vemurafenib',
  'but',
  'new',
  'scan',
  'shows',
  'growth',
  'bones',
  'and',
  'liver',
  'scanning',
  'brain',
  'this',
  'week'],
 ['i',
  'm',
  'getting',
  'transferred',
  'to',
  'washington',
  'u',
  'where',
  'my',
  'oncologist',
  'is',
  'and',
  'he',
  's',
  'out',
  'of',
  'town',
  'until',
  'd

In [72]:
len(words)

1500

In [140]:
A =[]
for i in words:
    a = " ".join(str(x) for x in i)
    A.append(a) 
  


In [146]:
string_words= ' '.join(str(x) for x in words)



In [331]:
from imblearn.over_sampling import SMOTE
ros = SMOTE(random_state=111)

ImportError: A sklearn version of at least 0.19.0 is required to use imbalanced-learn. 0.17.1 was found. Please upgrade sklearn

In [333]:

def balanced_subsample(x,y,subsample_size=1.0,random_state = 111):
    
    
    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        # if len(this_xs) > use_elems:
        #    this_xs = this_xs.reindex(np.random.permutation(this_xs.index))
        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = pd.concat(xs)
    ys = pd.Series(data=np.concatenate(ys),name='target')

    return xs,ys

   

In [295]:
xs = np.concatenate(xs)
ys = np.concatenate(ys)

In [334]:
from sklearn.cross_validation import train_test_split as tts
from scipy import sparse
random_state=111
x = posts["Tokenized.Sentences"]
y = posts["New_flag"]
X,Y = balanced_subsample(x,y)


X_train, X_test, y_train, y_test = tts(X,Y, test_size=0.2, random_state=111)
preprocessed_train = preprocessing(X_train)


string_words= ' '.join(map(str,preprocessed_train))
    
A =[]

for i in preprocessed_train:
    a = " ".join(str(x) for x in i)
    A.append(a) 
    



In [243]:
len(X_train)

603

In [249]:
print(len(np.where(y_train==1)[0]))
print(len(np.where(y_train==0)[0]))

297
306


In [336]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

#y = len(string_words.split())
#z = int(round(y*1))
vectorizer = TfidfVectorizer(use_idf= True)
#vectorizer= CountVectorizer()
X = vectorizer.fit_transform(A)
#indices = np.argsort(vectorizer)[::-1]
#features = vectorizer.get_feature_names()
#top_n = z
#top_features = [features[i] for i in indices[:top_n]]

type(X)
X

<654x2083 sparse matrix of type '<type 'numpy.float64'>'
	with 10042 stored elements in Compressed Sparse Row format>

In [155]:
top_features

[u'024360']

In [97]:
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
y = len(string_words.split())
z = int(round(y*0.4))
vectorizer = TfidfVectorizer(ngram_range=(1,3),use_idf=True)
#vectorizer= CountVectorizer()
X = vectorizer.fit_transform(A)
features_by_gram = defaultdict(list)

for f, w in zip(vectorizer.get_feature_names(), vectorizer.idf_):
    features_by_gram[len(f.split(' '))].append((f, w))
top_n = z
for gram, features in features_by_gram.iteritems():
    top_features = sorted(features, key=lambda x: x[1], reverse=True)[:top_n]
    top_features = [f[0] for f in top_features]
    

In [189]:
indices_list = []
for f in top_features:
    indices_list.append(vectorizer.vocabulary_[f])

[u'02 02 2017', u'02 2017 there', u'024360 trail from', u'10 19 2012']


In [170]:
# filter top_nfeatres for from complete sparse matrix
feature = X.tocsc()[:,indices_list] 


In [337]:
from sklearn.svm import LinearSVC

classifier = LinearSVC()
model = classifier.fit(X, y_train)

preprocessed_test = preprocessing(X_test)
B =[]
for i in preprocessed_test:
    a = " ".join(str(x) for x in i)
    B.append(a) 
    
tfs_test = vectorizer.transform(B)
feature_test = tfs_test

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()

y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

        0.0       0.70      0.83      0.76        76
        1.0       0.82      0.69      0.75        88

avg / total       0.77      0.76      0.76       164



In [44]:

#TempCount_test = TemporalEntitiesCount(X_test)

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()

y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

ValueError: X.shape[1] = 50 should be equal to 127, the number of features at training time

In [169]:
from sklearn import svm
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7).fit(Y_TRAIN,feature)


ValueError: bad input shape (739L, 7518L)